In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
from datetime import datetime
from dotenv import load_dotenv
from pathlib import Path

In [11]:
# .env 파일 로드
load_dotenv()

DOWNLOAD_TIMEOUT = 60  # 다운로드 대기 시간

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
download_prefs = {
    "download.default_directory": os.path.expanduser(os.getenv("DOWNLOAD_PATH")),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
options.add_experimental_option("prefs", download_prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)

try:
    # 페이지 열기 및 로그인
    driver.get(os.getenv("BASE_URL"))
    time.sleep(2)

    # 로그인 정보 입력
    userid_field = driver.find_element(By.ID, "userid")
    password_field = driver.find_element(By.ID, "passwd")

    userid_field.clear()
    password_field.clear()

    userid_field.send_keys(os.getenv("SELLMATE_ID"))
    password_field.send_keys(os.getenv("SELLMATE_PW"))

    driver.find_element(By.CLASS_NAME, "login_btn").click()
    time.sleep(2)

    # 주문 입력 페이지 접근
    driver.get(f"{os.getenv('BASE_URL')}/orders/input_order_Auto_web.asp")
    # JavaScript executor를 사용하여 체크박스 클릭
    for i in range(1, 8):
        try:
            # 체크박스 요소 찾기
            checkbox = driver.find_element(By.CLASS_NAME, f"check_{i}")

            # 체크박스가 선택되지 않은 경우에만 클릭
            if not checkbox.is_selected():
                # JavaScript를 사용하여 클릭 이벤트 실행
                driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(0.5)  # 클릭 후 잠시 대기

            # 요소가 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
            time.sleep(0.5)  # 스크롤 후 잠시 대기

        except Exception as e:
            print(f"체크박스 {i} 클릭 중 오류 발생: {str(e)}")
            continue

    # 선택된 항목 자동가져오기 버튼 찾기 및 클릭
    try:
        # 버튼 찾기 (onclick="multi_submit();" 속성을 가진 버튼)
        auto_import_button = driver.find_element(
            By.CSS_SELECTOR, "button[onclick='multi_submit();']"
        )

        # 버튼이 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView(true);", auto_import_button)
        time.sleep(1)  # 스크롤 후 잠시 대기

        # JavaScript를 사용하여 클릭
        driver.execute_script("arguments[0].click();", auto_import_button)

        print("자동가져오기 버튼 클릭 완료")

    except Exception as e:
        print(f"자동가져오기 버튼 클릭 중 오류 발생: {str(e)}")

    try:

        def check_progress_text(driver):
            progress_element = driver.find_element(By.ID, "progressText")
            return "주문입력 완료" in progress_element.text

        # 최대 300초 동안 "주문입력 완료" 텍스트가 나타날 때까지 대기
        WebDriverWait(driver, 300).until(check_progress_text)
        print("주문입력 완료")

    except TimeoutException:
        print("주문입력 완료 메시지가 300초 내에 나타나지 않았습니다.")
    except Exception as e:
        print(f"진행 상태 확인 중 오류 발생: {str(e)}")

    def handle_alerts():
        try:
            while True:
                try:
                    alert = driver.switch_to.alert
                    alert.accept()
                    time.sleep(1)
                except NoAlertPresentException:
                    break
        except Exception as e:
            print(f"Alert 처리 중 오류 발생: {str(e)}")

    handle_alerts()
    """중복주문 체크"""
    driver.get(f"{os.getenv('BASE_URL')}/orders/same_orderno.asp")
    # Alert가 있는지 확인하고 처리
    try:
        WebDriverWait(driver, 3).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()
    except TimeoutException:
        pass  # Alert가 없는 경우 진행
    wait.until(
        EC.presence_of_element_located((By.ID, "same_order_table"))
    )
    """자동 매칭"""
    today = datetime.now().strftime("%Y-%m-%d")
    driver.get(f"{os.getenv('BASE_URL')}/orders/code_matching_ok.asp?fromPage=orders&shop_code=all&date_type=o_date&mydate={today}&mydate2={today}&songFilter=all&searchPdnm=&searchOpt=&menu=%EC%9E%90%EB%8F%99%EB%A7%A4%EC%B9%AD(%EC%99%BC%EC%AA%BD%EB%A9%94%EB%89%B4)")
    wait.until(
        EC.presence_of_element_located((By.ID, "btn_all_matching"))
    )
    """묶음 정리"""
    # driver.get(f"{os.getenv('BASE_URL')}/orders/merging_order_ver2.asp?mydate={today}")
    driver.get(f"{os.getenv('BASE_URL')}/orders/merging_order_ok_ver2.asp?mydate={today}")
    wait.until(
        EC.presence_of_element_located((By.ID, "dataList"))
    )
    """재고할당"""
    # driver.get(f"{os.getenv('BASE_URL')}/orders/stk_apply_version_c.asp")
    driver.get(f"{os.getenv('BASE_URL')}/orders/stk_apply_version_c_ok.asp?opt_idx=&from=order&next_page=&response_type=&sub_manager=")
    wait.until(
        EC.presence_of_element_located((By.ID, "right-div"))
    )
    """취소주문체크"""
    driver.get(f"{os.getenv('BASE_URL')}/orders/cancel_order_Auto_web.asp")
    # JavaScript executor를 사용하여 체크박스 클릭
    for i in range(1, 8):  # check_1 부터 check_7 까지 (아몬드영 스마트스토어 ~ 인터파크)
        try:
            # 체크박스 요소 찾기
            checkbox = driver.find_element(By.CLASS_NAME, f"check_{i}")
            
            # 체크박스가 선택되지 않은 경우에만 클릭
            if not checkbox.is_selected():
                # JavaScript를 사용하여 클릭 이벤트 실행
                driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(0.5)  # 클릭 후 잠시 대기
                
        except Exception as e:
            print(f"체크박스 {i} 클릭 중 오류 발생: {str(e)}")
            continue

    time.sleep(1)  # 모든 체크박스 선택 후 잠시 대기

    # "선택된 항목 자동 취소주문체크" 버튼 찾기 및 클릭
    try:
        # 네이버 버튼 찾기 (onclick="multi_submit();" 속성을 가진 버튼)
        auto_check_button = driver.find_element(By.CSS_SELECTOR, "button[onclick='porting_cancel_check_all();']")
        
        # 버튼이 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView(true);", auto_check_button)
        time.sleep(1)  # 스크롤 후 잠시 대기
        
        # JavaScript를 사용하여 클릭
        driver.execute_script("arguments[0].click();", auto_check_button)
        
        print("취소주문 체크 버튼 클릭 완료")
        
    except Exception as e:
        print(f"취소주문 체크 버튼 클릭 중 오류 발생: {str(e)}")

    # 작업 완료 대기 (필요한 경우)
    time.sleep(5)

finally:
    # 브라우저 닫기
    print("hello")

체크박스 1 클릭 중 오류 발생: Alert Text: 잠깐만요! 지금 데이터를 안전하게 처리 하는 중입니다.  잠시 후 다시 시도해 주세요.
작업자id: 01064759138 | 홀딩시간 : 6초 (600초)
메뉴 : 주문관리 > 주문입력
Message: unexpected alert open: {Alert text : 잠깐만요! 지금 데이터를 안전하게 처리 하는 중입니다.  잠시 후 다시 시도해 주세요.
작업자id: 01064759138 | 홀딩시간 : 6초 (600초)
메뉴 : 주문관리 > 주문입력}
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7B57FFB05+28789]
	(No symbol) [0x00007FF7B57686E0]
	(No symbol) [0x00007FF7B560592A]
	(No symbol) [0x00007FF7B56A01F4]
	(No symbol) [0x00007FF7B567F1E3]
	(No symbol) [0x00007FF7B564A938]
	(No symbol) [0x00007FF7B564BAA1]
	GetHandleVerifier [0x00007FF7B5B3933D+3410093]
	GetHandleVerifier [0x00007FF7B5B4E7DD+3497293]
	GetHandleVerifier [0x00007FF7B5B42A73+3448803]
	GetHandleVerifier [0x00007FF7B58C7BBB+848171]
	(No symbol) [0x00007FF7B5773C3F]
	(No symbol) [0x00007FF7B576F6E4]
	(No symbol) [0x00007FF7B576F87D]
	(No symbol) [0x00007FF7B575ED49]
	BaseThreadInitThunk [0x00007FF84F9DE8D7+23]
	RtlUserThreadStart [0x00007FF850A5FBCC+

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7B57FFB05+28789]
	(No symbol) [0x00007FF7B57686E0]
	(No symbol) [0x00007FF7B560592A]
	(No symbol) [0x00007FF7B55DF505]
	(No symbol) [0x00007FF7B5686477]
	(No symbol) [0x00007FF7B569EF42]
	(No symbol) [0x00007FF7B567F1E3]
	(No symbol) [0x00007FF7B564A938]
	(No symbol) [0x00007FF7B564BAA1]
	GetHandleVerifier [0x00007FF7B5B3933D+3410093]
	GetHandleVerifier [0x00007FF7B5B4E7DD+3497293]
	GetHandleVerifier [0x00007FF7B5B42A73+3448803]
	GetHandleVerifier [0x00007FF7B58C7BBB+848171]
	(No symbol) [0x00007FF7B5773C3F]
	(No symbol) [0x00007FF7B576F6E4]
	(No symbol) [0x00007FF7B576F87D]
	(No symbol) [0x00007FF7B575ED49]
	BaseThreadInitThunk [0x00007FF84F9DE8D7+23]
	RtlUserThreadStart [0x00007FF850A5FBCC+44]
